In this third implementation of the Xception architecture, we try to tune further the data augmentation. The idea behind it was that by changing even the brightness of the images we could obtain similar results to the ill leaves, improving our score.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

/kaggle/input/dataset-leaves/training/Orange/05938.jpg
/kaggle/input/dataset-leaves/training/Orange/06122.jpg
/kaggle/input/dataset-leaves/training/Orange/07457.jpg
/kaggle/input/dataset-leaves/training/Orange/06957.jpg
/kaggle/input/dataset-leaves/training/Orange/07015.jpg
/kaggle/input/dataset-leaves/training/Orange/06572.jpg
/kaggle/input/dataset-leaves/training/Orange/07329.jpg
/kaggle/input/dataset-leaves/training/Orange/07448.jpg
/kaggle/input/dataset-leaves/training/Orange/05877.jpg
/kaggle/input/dataset-leaves/training/Orange/06922.jpg
/kaggle/input/dataset-leaves/training/Orange/05754.jpg
/kaggle/input/dataset-leaves/training/Orange/06431.jpg
/kaggle/input/dataset-leaves/training/Orange/05841.jpg
/kaggle/input/dataset-leaves/training/Orange/07587.jpg
/kaggle/input/dataset-leaves/training/Orange/07405.jpg
/kaggle/input/dataset-leaves/training/Orange/06563.jpg
/kaggle/input/dataset-leaves/training/Orange/06712.jpg
/kaggle/input/dataset-leaves/training/Orange/06909.jpg
/kaggle/in

In [ ]:
# Random seed for reproducibility

n_classes = 14
seed = 22 #Gonzales o Chiesa

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

dataset_dir = '/kaggle/input/dataset-leaves/training'

In [ ]:
!pip install split_folders
import splitfolders

In [ ]:
splitfolders.ratio(dataset_dir, output="input/dataset", seed=seed, ratio=(0.85, 0.15)) 

Copying files: 17728 files [00:42, 417.92 files/s]


The following part shows the new augmentation:

In [ ]:
train_dir = "input/dataset/train"
val_dir = "input/dataset/val"

# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Create an instance of ImageDataGenerator for training, validation, and test sets
aug_train_data_gen = ImageDataGenerator(rotation_range=45,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        brightness_range = (-0.5, 0.5),
                                        fill_mode='constant', cval = 0,
                                        rescale=1/255.) # rescale value is multiplied to the image
val_data_gen = ImageDataGenerator(rescale=1/255.)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=train_dir,
                                                       target_size=(256,256),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=8,
                                                       shuffle=True,
                                                       seed=seed)
val_gen = val_data_gen.flow_from_directory(directory=val_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=False,
                                               seed=seed)


Found 15062 images belonging to 14 classes.
Found 2666 images belonging to 14 classes.


In [ ]:
input_shape = (256, 256, 3)
epochs = 60

In [ ]:
def entry_flow(inputs):
  # Entry block
  x = tfkl.Conv2D(32, 3, strides=2, padding='same')(inputs)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.Activation('relu')(x)

  x = tfkl.Conv2D(64, 3, padding='same')(x)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.Activation('relu')(x)

  previous_block_activation = x  # Set aside residual
  
  # Blocks 1, 2, 3 are identical apart from the feature depth.
  for size in [128, 256, 728]:
    x = tfkl.Activation('relu')(x)
    x = tfkl.SeparableConv2D(size, 3, padding='same')(x)
    x = tfkl.BatchNormalization()(x)

    x = tfkl.Activation('relu')(x)
    x = tfkl.SeparableConv2D(size, 3, padding='same')(x)
    x = tfkl.BatchNormalization()(x)

    x = tfkl.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Project residual
    residual = tfkl.Conv2D(
        size, 1, strides=2, padding='same')(previous_block_activation)
    x = tfkl.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual

  return x

def middle_flow(x, num_blocks=8):
  
  previous_block_activation = x

  for _ in range(num_blocks):
    x = tfkl.Activation('relu')(x)
    x = tfkl.SeparableConv2D(728, 3, padding='same')(x)
    x = tfkl.BatchNormalization()(x)

    x = tfkl.Activation('relu')(x)
    x = tfkl.SeparableConv2D(728, 3, padding='same')(x)
    x = tfkl.BatchNormalization()(x)
    
    x = tfkl.Activation('relu')(x)
    x = tfkl.SeparableConv2D(728, 3, padding='same')(x)
    x = tfkl.BatchNormalization()(x)

    x = tfkl.add([x, previous_block_activation])  # Add back residual
    previous_block_activation = x  # Set aside next residual
    
  return x

def exit_flow(x, num_classes=n_classes):

  previous_block_activation = x

  x = tfkl.Activation('relu')(x)
  x = tfkl.SeparableConv2D(728, 3, padding='same')(x)
  x = tfkl.BatchNormalization()(x)

  x = tfkl.Activation('relu')(x)
  x = tfkl.SeparableConv2D(1024, 3, padding='same')(x)
  x = tfkl.BatchNormalization()(x)
  
  x = tfkl.MaxPooling2D(3, strides=2, padding='same')(x)

  # Project residual
  residual = tfkl.Conv2D(
      1024, 1, strides=2, padding='same')(previous_block_activation)
  x = tfkl.add([x, residual])  # Add back residual
  
  x = tfkl.SeparableConv2D(1536, 3, padding='same')(x)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.Activation('relu')(x)
  
  x = tfkl.SeparableConv2D(2048, 3, padding='same')(x)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.Activation('relu')(x)
  
  x = tfkl.GlobalAveragePooling2D()(x)
  if num_classes == 1:
    activation = 'sigmoid'
  else:
    activation = 'softmax'
  return tfkl.Dense(num_classes, activation=activation)(x)

In [ ]:
# Create Xception by chaining the 3 flows
inputs = tfk.Input(shape=(256, 256, 3))
outputs = exit_flow(middle_flow(entry_flow(inputs)))
model = tfk.Model(inputs, outputs)
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
model.summary()

2021-11-19 11:35:12.597602: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 11:35:12.699850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 11:35:12.700575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 11:35:12.702370: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('results')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [ ]:
# Create folders and callbacks and fit
aug_callbacks = create_folders_and_callbacks(model_name='Xception_Aug3')

# Train the model
history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data = val_gen,
    callbacks = aug_callbacks,
).history

model.save("models/Xception_Aug3")

2021-11-19 11:35:16.558628: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/60


2021-11-19 11:35:21.651136: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1883/1883 [==============================] - 369s 191ms/step - loss: 2.3622 - accuracy: 0.3130 - val_loss: 3.6187 - val_accuracy: 0.2236
Epoch 2/60
1883/1883 [==============================] - 362s 192ms/step - loss: 2.2548 - accuracy: 0.3290 - val_loss: 3.4768 - val_accuracy: 0.3252
Epoch 3/60
1883/1883 [==============================] - 369s 196ms/step - loss: 2.1883 - accuracy: 0.3458 - val_loss: 2.4951 - val_accuracy: 0.3140
Epoch 4/60
1883/1883 [==============================] - 368s 195ms/step - loss: 2.1491 - accuracy: 0.3567 - val_loss: 3.5351 - val_accuracy: 0.4355
Epoch 5/60
1883/1883 [==============================] - 368s 195ms/step - loss: 2.0833 - accuracy: 0.3744 - val_loss: 2.0156 - val_accuracy: 0.5083
Epoch 6/60
1883/1883 [==============================] - 371s 197ms/step - loss: 1.9957 - accuracy: 0.3955 - val_loss: 1.8632 - val_accuracy: 0.5349
Epoch 7/60
1883/1883 [==============================] - 366s 194ms/step - loss: 1.9102 - accuracy: 0.4217 - val_loss: 2.147

2021-11-19 15:48:15.050382: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
print("Training complete!")

Training complete!
